## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [3]:
def submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid,ars_url='https://ars-dev.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [4]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            file_read = path.join(root, name)
            dir_name = (os.path.splitext(os.path.basename(root))[0])
            print(file_read)
            
            filename = (os.path.splitext(os.path.basename(file_read))[0])
            print(filename)
            with open(file_read,'r') as inf:
                query = json.load(inf)
                
                kcresult = submit_to_ars(query)
                
                sleep(300)
                
                result_status = retrieve_ars_results(kcresult)
                
        
                dict_workflows[filename] = kcresult
                            
                sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=44922669-4ad3-449b-9edf-8edb2ab401a8
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 74
Error
ara-bte Error 0
Unknown
ara-ncats Unknown 0
Done
ara-robokop Done 30
Done
ara-unsecret Done 99
Done
kp-genetics Done 0
Done
kp-molecular Done 99
Done
ara-explanatory Done 0
Done
ara-improving Done 5
Error
kp-cam Error 0
Done
kp-textmining Done 14
Error
kp-molecular Error 0
Done
kp-openpredict Done 0
Done
kp-cohd ARS Error 0
Error
ara-aragorn-exp Error 0
Done
kp-chp Done 20
Error
kp-icees Error 0
Done
kp-icees-dili Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.ncats.io/?source=ARS&id=b7f401db-0040-4906-b8e2-b3c8a675b0de
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax ARS Error 0
Error
ara-bte Error 0
Unknown
ara-ncats Unknown 0
Done
ara-robokop Done 83
Done
ara-unsecret Done 0
D

ara-robokop Error 0
Done
ara-unsecret Done 0
Error
kp-genetics Error 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Error
ara-improving Error 0
Error
kp-cam Error 0
Done
kp-textmining Done 0
Error
kp-molecular Error 0
Error
kp-openpredict Error 0
Error
kp-cohd Error 0
Error
ara-aragorn-exp Error 0
Done
kp-chp Done 0
Error
kp-icees Error 0
Error
kp-icees-dili Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.3_DILI-one-hop-from-genes.json
B.3_DILI-one-hop-from-genes
https://arax.ncats.io/?source=ARS&id=49d36e61-9137-4983-b852-8a02af40b124
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 76
Error
ara-bte Error 0
Unknown
ara-ncats Unknown 0
Done
ara-robokop Done 21
Done
ara-unsecret Done 0
Done
kp-genetics Done 0
Error
kp-molecular Error 0
Done
ara-explanatory Done 0
Done
ara-improving Done 874
Error
kp-cam Error 0
Done
kp-textmining Done 0
Error
kp-molecular Error 0
Done
kp-openpredict Done 0
Done
kp-cohd ARS Error 0
Error
ara-aragorn-exp Err

kp-chp Done 0
Error
kp-icees Error 0
Done
kp-icees-dili Done 0


<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [5]:
dict_workflows

{'A.3_KCNMA1': '44922669-4ad3-449b-9edf-8edb2ab401a8',
 'A.8_EGFR_simple': 'b7f401db-0040-4906-b8e2-b3c8a675b0de',
 'A.0_RHOBTB2_direct': 'd4c8cbc0-30ce-493d-b01a-d2011e0bf621',
 'A.2_RHOBTB2_twohop': '709582ed-422d-4682-9d94-baa3dc1d7df8',
 'A.9_EGFR_advanced': '6c919c51-43b1-48de-94e6-a7e0e893df33',
 'A.1_RHOBTB2': 'c55ff2cc-950e-4bbb-8673-26530e09a2ea',
 'B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi': 'add5bdf7-5af4-4349-abde-0af630e7067c',
 'B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi': '27f95f00-4d6b-4569-acc6-509e276aae5f',
 'B.4_one-hop-gene-biological-process-or-activity_trapi': '111ab2e9-7d9d-4025-8459-ff808bc8e887',
 'B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi': '78112595-d314-4e25-a927-81af92945686',
 'B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi': '99fe2f81-2c10-42a0-a980-5cef1ea66b96',
 'B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi': '35153cca-f7a3-41f8-818a-f8aafa4cb3fc',
 'B.3_DILI-one

In [6]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1 44922669-4ad3-449b-9edf-8edb2ab401a8
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 74
Error
ara-bte Error 0
Unknown
ara-ncats Unknown 0
Done
ara-robokop Done 30
Done
ara-unsecret Done 99
Done
kp-genetics Done 0
Done
kp-molecular Done 99
Done
ara-explanatory Done 0
Done
ara-improving Done 5
Error
kp-cam Error 0
Done
kp-textmining Done 14
Error
kp-molecular Error 0
Done
kp-openpredict Done 0
Done
kp-cohd ARS Error 0
Error
ara-aragorn-exp Error 0
Done
kp-chp Done 20
Error
kp-icees Error 0
Done
kp-icees-dili Done 0
A.8_EGFR_simple b7f401db-0040-4906-b8e2-b3c8a675b0de
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax ARS Error 0
Error
ara-bte Error 0
Unknown
ara-ncats Unknown 0
Done
ara-robokop Done 83
Done
ara-unsecret Done 0
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Done
ara-explanatory Done 0
Done
ara-improving Done 26
Error
kp-cam Error 0
Done
kp-textmining Done 88
Error
kp-molecular Error 0
Done
kp-openpredict Done 0
Error
kp-cohd Error 0
Error
ara-aragorn-exp Er

kp-molecular Error 0
Done
ara-explanatory Done 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 0
Done
kp-textmining Done 0
Error
kp-molecular Error 0
Done
kp-openpredict Done 0
Error
kp-cohd Error 0
Error
ara-aragorn-exp Error 0
Done
kp-chp Done 0
Error
kp-icees Error 0
Done
kp-icees-dili ARS Error 0
B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi 96bb6e8e-2365-4c47-9528-1752e66f6a70
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 0
Error
ara-bte Error 0
Unknown
ara-ncats Unknown 0
Error
ara-robokop Error 0
Done
ara-unsecret Done 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Done
ara-explanatory Done 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 0
Done
kp-textmining Done 0
Error
kp-molecular Error 0
Done
kp-openpredict Done 0
Error
kp-cohd Error 0
Error
ara-aragorn-exp Error 0
Error
kp-chp Error 0
Error
kp-icees Error 0
Done
kp-icees-dili ARS Error 0
D.1_parkinsons-crohns b77467b0-f271-4c31-8836-080a52928450
Done
Done
ara-aragorn ARS Error 0
Done
ara

### Creating dataframe for workflows with PK

<br>

In [135]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)

for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][0].items():
        #print(key, value)
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.0_Initial_ARS_three-hop_query_results_MESH:D056487
B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi
B.3_DILI-one-hop-from-genes
B.4_one-hop-gene-biological-process-or-activity_trapi
C.1_ChemSubstances_related_to_Disease
C.2_ChemSubstances_related_to_GeneSet
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine
D.6_metformin-ferritin


In [136]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=d4c8cbc0-...,https://arax.ncats.io/?source=ARS&id=c55ff2cc-...,https://arax.ncats.io/?source=ARS&id=709582ed-...,https://arax.ncats.io/?source=ARS&id=44922669-...,https://arax.ncats.io/?source=ARS&id=b7f401db-...,https://arax.ncats.io/?source=ARS&id=6c919c51-...,https://arax.ncats.io/?source=ARS&id=7f0e757f-...,https://arax.ncats.io/?source=ARS&id=add5bdf7-...,https://arax.ncats.io/?source=ARS&id=27f95f00-...,https://arax.ncats.io/?source=ARS&id=3bae1bd7-...,...,https://arax.ncats.io/?source=ARS&id=35153cca-...,https://arax.ncats.io/?source=ARS&id=49d36e61-...,https://arax.ncats.io/?source=ARS&id=111ab2e9-...,https://arax.ncats.io/?source=ARS&id=d6fe8a1f-...,https://arax.ncats.io/?source=ARS&id=73fe529b-...,https://arax.ncats.io/?source=ARS&id=b77467b0-...,https://arax.ncats.io/?source=ARS&id=9415ea82-...,https://arax.ncats.io/?source=ARS&id=95a4c4d7-...,https://arax.ncats.io/?source=ARS&id=387d368e-...,https://arax.ncats.io/?source=ARS&id=f93b1407-...
ara-arax,No Results: 200,Results: 200,ARS Error: 200,Results: 200,ARS Error: 200,ARS Error: 200,No Results: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,...,Error: 400,Results: 200,Results: 200,ARS Error: 200,Results: 200,ARS Error: 200,ARS Error: 200,Results: 200,Results: 200,ARS Error: 200
ara-aragorn,No Results: 200,ARS Error: 200,No Results: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,No Results: 200,ARS Error: 200,No Results: 200,...,Error: 422,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,No Results: 200,ARS Error: 200
ara-bte,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504
kp-genetics,No Results: 200,No Results: 200,Unknown: 503,No Results: 200,No Results: 200,No Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Error: 400,No Results: 200,No Results: 200,Error: 504,Error: 504,Unknown: 503,Error: 504,Error: 504,Error: 504,Error: 504
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 20

In [137]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [138]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=d4c8cbc0-...,https://arax.ncats.io/?source=ARS&id=c55ff2cc-...,https://arax.ncats.io/?source=ARS&id=709582ed-...,https://arax.ncats.io/?source=ARS&id=44922669-...,https://arax.ncats.io/?source=ARS&id=b7f401db-...,https://arax.ncats.io/?source=ARS&id=6c919c51-...,https://arax.ncats.io/?source=ARS&id=7f0e757f-...,https://arax.ncats.io/?source=ARS&id=add5bdf7-...,https://arax.ncats.io/?source=ARS&id=27f95f00-...,https://arax.ncats.io/?source=ARS&id=3bae1bd7-...,...,https://arax.ncats.io/?source=ARS&id=35153cca-...,https://arax.ncats.io/?source=ARS&id=49d36e61-...,https://arax.ncats.io/?source=ARS&id=111ab2e9-...,https://arax.ncats.io/?source=ARS&id=d6fe8a1f-...,https://arax.ncats.io/?source=ARS&id=73fe529b-...,https://arax.ncats.io/?source=ARS&id=b77467b0-...,https://arax.ncats.io/?source=ARS&id=9415ea82-...,https://arax.ncats.io/?source=ARS&id=95a4c4d7-...,https://arax.ncats.io/?source=ARS&id=387d368e-...,https://arax.ncats.io/?source=ARS&id=f93b1407-...
ara-arax,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200
ara-aragorn,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-bte,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504
kp-genetics,No Results: 200,No Results: 200,Unknown: 503,No Results: 200,No Results: 200,No Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Error: 400,No Results: 200,No Results: 200,Error: 504,Error: 504,Unknown: 503,Error: 504,Error: 504,Error: 504,Error: 504
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Res

### Creating second table with edge attribute source

<br>

In [ ]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

In [ ]:
final_dict2

In [ ]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [ ]:
final_dictassemble

In [ ]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [ ]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [ ]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [ ]:
df2test = pd.DataFrame(df2test.unstack().T)

In [ ]:
df2test

In [ ]:
#df2test.drop([''], axis=0, inplace=True)

In [ ]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [ ]:
df2test

In [ ]:
#df2.replace([], 'None', regex=True,inplace=True)

In [ ]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [ ]:
d
f2test = df2test.rename_axis(None)

In [ ]:
df2test.columns.name = None

In [ ]:
df2test.fillna('', inplace=True)

In [ ]:
df2test

In [ ]:
pwd

<br>

## Converting the Pk's to hyperlink

<br>

In [139]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [140]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [141]:
df.sort_index(inplace=True)

In [142]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200
ara-bte,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502


In [143]:
df['Query Type'] = 'Async'

df = df[['Query Type']+ list(df.columns[:-1])]

In [144]:
df

,Query Type,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Async,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-aragorn-exp,Async,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,Async,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200
ara-bte,Async,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-explanatory,Async,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Async,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-ncats,Async,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,Async,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,Async,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504
kp-cam,Async,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502


<br>

### Highlight the cells In Excel

<br>

In [15]:
def highlight(v):
    if v.startswith('Results'):
        return 'background-color: %s' % 'green'
    elif v.startswith('Error'):
        return 'background-color: %s' % 'red'
    elif v.startswith('No Results'):
        return 'background-color: %s' % 'yellow'
    elif v.startswith('ARS Error'):
        return 'background-color: %s' % 'blue'
    elif v.startswith('Unknown'):
        return 'background-color: %s' % 'magneta'
    else:
        return

In [16]:
styled = df.style.applymap(highlight)

In [17]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 422,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200
ara-bte,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 500,No Results: 200,No Results: 200,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error:

In [ ]:
#import xlsxwriter

In [ ]:
#import xlwt

In [18]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker Asynchronous_' + date

In [19]:
wks_name

'Workflow Progress Tracker Asynchronous_2021_09_15-12_02_13_AM'

In [ ]:
#writer = pd.ExcelWriter('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx', engine='xlsxwriter')

In [ ]:
#styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx')

#styled.to_excel(writer, sheet_name='progress_tracker')
#df2.to_excel(writer, sheet_name='edge_attribute')

In [ ]:
### Save as excel sheet
#df.to_excel('ara_worklow_progress_tracker_Aug5.xlsx', sheet_name = 'Progress_Tracker')

In [ ]:
### save all the ids for ara submission
#with open('ara_worklow_IDs_Aug9.txt', 'w') as f:
#    print(dict_workflows, file=f)

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [20]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 422,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200
ara-bte,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 500,No Results: 200,No Results: 200,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error:

In [67]:
df

,index,Query Type,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,...,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
0,ara-aragorn,Async,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
1,ara-aragorn-exp,Async,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
2,ara-arax,Async,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200
3,ara-bte,Async,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
4,ara-explanatory,Async,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
5,ara-improving,Async,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
6,ara-ncats,Async,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
7,ara-robokop,Async,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,...,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
8,ara-unsecret,Async,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504
9,kp-cam,Async,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502


In [64]:
df.reset_index(inplace=True)

In [111]:
df.rename({'index': 'ARA/KP'}, axis=1, inplace=True)

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [69]:
df

,ARA/KP,Query Type,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,...,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
0,ara-aragorn,Async,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
1,ara-aragorn-exp,Async,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
2,ara-arax,Async,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200
3,ara-bte,Async,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
4,ara-explanatory,Async,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
5,ara-improving,Async,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
6,ara-ncats,Async,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
7,ara-robokop,Async,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,...,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
8,ara-unsecret,Async,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504
9,kp-cam,Async,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502


<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [112]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *
import gspread_dataframe as gd

In [123]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [175]:
ws = gc.open("workflow_progress_tracker").worksheet("TESTING ONLY")

In [174]:
#ws.resize(19,24)

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

In [176]:
existing = gd.get_as_dataframe(ws)

In [177]:
existing

,Unnamed: 0,Query Type,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,...,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
0,ara-aragorn,Sync,No Results: 200,Error: 500,No Results: 200,Error: 500,Error: 500,Error: 500,Error: 500,No Results: 200,...,Error: 422,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,No Results: 200,Error: 500
1,ara-aragorn-exp,Sync,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
2,ara-arax,Sync,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,...,Error: 400,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598
3,ara-bte,Sync,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,...,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Error: 598,Results: 200
4,ara-explanatory,Sync,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
5,ara-improving,Sync,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
6,ara-robokop,Sync,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,...,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
7,ara-unsecret,Sync,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,Error: 504
8,kp-cam,Sync,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502
9,kp-chp,Sync,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 502,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200


In [178]:
existing.set_index('Unnamed: 0', inplace=True)

In [179]:
df.shape

(19, 24)

In [180]:
existing.shape

(18, 24)

In [181]:
existing = existing.rename_axis(None)

In [182]:
existing

,Query Type,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Sync,No Results: 200,Error: 500,No Results: 200,Error: 500,Error: 500,Error: 500,Error: 500,No Results: 200,Error: 598,...,Error: 422,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,No Results: 200,Error: 500
ara-aragorn-exp,Sync,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,Sync,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,...,Error: 400,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598
ara-bte,Sync,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Error: 598,Results: 200
ara-explanatory,Sync,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Sync,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,Sync,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,Sync,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,Error: 504
kp-cam,Sync,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502
kp-chp,Sync,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 502,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200


In [183]:
updated = existing.append(df)
gd.set_with_dataframe(ws, updated,include_index=True,include_column_header=True,resize=True)

In [184]:
#spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
#d2g.upload(df, spreadsheet_key, "TESTING ONLY", credentials=credentials, row_names=True)

In [188]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
wksh = gc.open("workflow_progress_tracker")
#sh = wksh.worksheet(wks_name)
sh = wksh.worksheet("TESTING ONLY")

In [189]:
ws

<Worksheet 'TESTING ONLY' id:526619635>

In [190]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}38', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

In [191]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}38', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 526619635,
       'startRowIndex': 1,
       'endRowIndex': 38,
       'startColumnIndex': 2,
       'endColumnIndex': 25}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {},
  {}]}

In [192]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}38', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 526619635,
       'startRowIndex': 1,
       'endRowIndex': 38,
       'startColumnIndex': 2,
       'endColumnIndex': 25}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 526619635,
       'startRowIndex': 1,
       'endRowIndex': 38,
       'startColumnIndex': 2,
       'endColumnIndex': 25}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}

In [193]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}38', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['ARS Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 526619635,
       'startRowIndex': 1,
       'endRowIndex': 38,
       'startColumnIndex': 2,
       'endColumnIndex': 25}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7490196,
         'green': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 526619635,
       'startRowIndex': 1,
       'endRowIndex': 38,
       'startColumnIndex': 2,
       'endColumnIndex': 25}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': Tru

In [194]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}38', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 526619635,
       'startRowIndex': 1,
       'endRowIndex': 38,
       'startColumnIndex': 2,
       'endColumnIndex': 25}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'ARS Error'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 526619635,
       'startRowIndex': 1,
       'endRowIndex': 38,
       'startColumnIndex': 2,
       'endColumnIndex': 25}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'text

In [195]:
sh

<Worksheet 'TESTING ONLY' id:526619635>

In [196]:
# Select a range
cell_list = sh.range('B19:Y19')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'TESTING ONLY'!B19:Y19",
 'updatedRows': 1,
 'updatedColumns': 24,
 'updatedCells': 24}

In [197]:
# Select a range
cell_list = sh.range('B38:Y38')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'TESTING ONLY'!B38:Y38",
 'updatedRows': 1,
 'updatedColumns': 24,
 'updatedCells': 24}

### Push Dataframe 2

<br>

In [ ]:
wks2 = 'edge_attribute_source_' + date

In [ ]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df2test, spreadsheet_key, wks2, credentials=credentials, row_names=True)